In [1]:
! pip install streamlit
!pip install langchain-google-genai
!pip install langchain langchain-google-genai google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.2 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached langchain_google_genai-2.0.11-py3-none-any.whl.metadata (3.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.4 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.16
    Uninstalling google-ai-generativelanguage-0.6.16:
      Successfully uninstalle

In [2]:
%%writefile tourplanner.py
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser

# Load API Key
with open("/content/code.txt") as f:
    API_KEY = f.read()

# Define AI Travel Assistant Prompt
# Define AI Travel Assistant Prompt
chat_template = ChatPromptTemplate(
    messages=[
        (
            "system",
            """
            You are an AI-powered travel assistant that provides users with the best travel options from a given source to a specified destination.
            Analyze multiple travel modes: flights, trains, buses, and cabs, and return details including:

            - Travel mode (Flight, Train, Bus, Cab)
            - Cost (in INR, prefixed with "₹")
            - Duration (e.g., "5 hours 30 minutes")
            - Trip details (Maximum 50 words)

            Ensure the response is a valid JSON **list** (not an object), structured like this:
            ```json
            {{
                "travel_options": [
                    {{
                        "travel_mode": "Flight",
                        "cost": "₹4500",
                        "duration": "2 hours",
                        "details": "Non-stop flight with free meal"
                    }},
                    {{
                        "travel_mode": "Train",
                        "cost": "₹1200",
                        "duration": "5 hours 30 minutes",
                        "details": "AC sleeper class available"
                    }}
                ]
            }}
            ```
            """
        ),
        ("human", "Find the best travel options from {source} to {destination}.")
    ]
)

# Initialize AI Model
chat_model = ChatGoogleGenerativeAI(google_api_key=API_KEY, model="gemini-1.5-flash")

# Define Output Parser
chat_parser = JsonOutputParser()

# Processing Chain
chain = chat_template | chat_model | chat_parser

# Function to Fetch Travel Recommendations
def get_travel_recommendations(source, destination):
    try:
        response = chain.invoke({"source": source, "destination": destination})

        # Ensure the response is in dictionary format
        if isinstance(response, str):
            response = chat_parser.parse(response)  # Parse using LangChain's parser

        # Extract the travel options if returned in a dictionary format
        if isinstance(response, dict) and "travel_options" in response:
            response = response["travel_options"]

        if isinstance(response, list) and all(isinstance(item, dict) for item in response):
            return response  # Valid travel options
        else:
            return {"error": "Invalid JSON format returned by AI. Please try again."}
    except Exception as e:
        return {"error": f"AI response error: {str(e)}"}


# Streamlit UI Setup
st.set_page_config(page_title="AI-Powered Travel Planner", page_icon="🌍", layout="centered")

st.title("🌍 AI-Powered Travel Planner")
st.subheader("Find the best travel options!")

# User Input Fields
source = st.text_input("Enter Source Location", placeholder="Departure City")
destination = st.text_input("Enter Destination Location", placeholder="Arrival City")

# Fetch and Display Travel Options
if st.button("Find Best Travel Options", use_container_width=True):
    if source and destination:
        with st.spinner("Fetching travel recommendations..."):
            result = get_travel_recommendations(source, destination)

            if isinstance(result, list):
                st.write("### 🚀 Recommended Travel Options:")

                for option in result:
                    travel_mode = option.get("travel_mode", "Unknown")
                    cost = option.get("cost", "N/A")
                    duration = option.get("duration", "Unknown")
                    details = option.get("details", "No details available")

                    st.markdown(f"### ✈ {travel_mode}")
                    st.markdown(f"💰 **Cost:** {cost}")
                    st.markdown(f"⏳ **Duration:** {duration}")
                    st.info(details)
                    st.divider()
            elif "error" in result:
                st.error(result["error"])
            else:
                st.error("Unexpected response format. Please try again.")
    else:
        st.warning("⚠ Please enter both source and destination locations.")



Writing tourplanner.py


In [ ]:
!npm install localtunnel
!streamlit run /content/tourplanner.py &>/content/logs.txt &
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹34.169.0.57
⠙your url is: https://fluffy-foxes-fix.loca.lt
